In [33]:
import pandas as pd 
import os
from dotenv import load_dotenv
import berserk
pd.get_option("display.max_colwidth", 200) 
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

# load content of env
load_dotenv()

# load the token
lichess_token = os.getenv("my_token")

# using token 
session = berserk.TokenSession(lichess_token)
client = berserk.Client(session=session)

In [56]:
# function to get top 10 based on type of chess
top10_players = client.users.get_all_top_10()

def get_top10_players(chess_type):
    if chess_type != "blitz":
        top_players = pd.json_normalize(top10_players, record_path=chess_type).rename(columns=
                {f"perfs.{chess_type}.rating" : f"{chess_type}_rating", f"perfs.{chess_type}.progress" : f"{chess_type}_progress"}).drop(
                    columns=["patron", "online"]
                )
    else : 
        top_players = pd.json_normalize(top10_players, record_path=chess_type).rename(columns=
                {f"perfs.{chess_type}.rating" : f"{chess_type}_rating", f"perfs.{chess_type}.progress" : f"{chess_type}_progress"}).drop(
                    columns=["online"]
                )
    return top_players

In [57]:
# top bullet 
top10_bullet = get_top10_players("bullet")
top10_bullet

,id,username,bullet_rating,bullet_progress,title
0,chesswarrior31,Chesswarrior31,3301,21,NaN
1,nihalsarin2004,nihalsarin2004,3265,-11,GM
2,muisback,muisback,3248,14,GM
3,azzaro25,Azzaro25,3247,38,GM
4,rebeccaharris,RebeccaHarris,3239,-52,GM
5,zhigalko_sergei,Zhigalko_Sergei,3218,9,GM
6,watneg,Watneg,3195,12,GM
7,mutdpro,mutdpro,3191,-14,GM
8,ediz_gurel,Ediz_Gurel,3173,3,GM
9,arm-777777,ARM-777777,3160,30,GM


In [58]:
# top 10 blitz
top10_blitz = get_top10_players("blitz")
top10_blitz

,id,username,title,blitz_rating,blitz_progress
0,cutemouse83,cutemouse83,GM,3018,29
1,dr-cro,Dr-CRO,GM,2960,10
2,lintchevski_daniil,Lintchevski_Daniil,GM,2956,10
3,athena-pallada,athena-pallada,GM,2951,-3
4,may6enexttime,may6enexttime,GM,2941,-39
5,ck-king-2024,CK-king-2024,NaN,2914,15
6,gsvc,gsvc,GM,2903,42
7,howardxue,HowardXue,NaN,2902,-13
8,silence404,silence404,NaN,2900,8
9,godson69,GODSON69,NaN,2900,-27


In [59]:
# top 10 rapid
top10_rapid = get_top10_players("rapid")
top10_rapid

,id,username,title,rapid_rating,rapid_progress
0,drvitman,DrVitman,GM,2929,-21
1,mysterious_expert,mysterious_expert,IM,2814,0
2,roman_yanchenko,Roman_Yanchenko,FM,2804,27
3,tuzakli_egitim,Tuzakli_Egitim,FM,2764,-39
4,kurald_galain,Kurald_Galain,NaN,2757,6
5,savvavetokhin2009,SavvaVetokhin2009,IM,2751,-12
6,rakhmanov_aleksandr,Rakhmanov_Aleksandr,GM,2751,-38
7,jimmy0613,jimmy0613,FM,2746,-3
8,konnov_oleg,Konnov_Oleg,NaN,2740,14
9,aborigen100500,Aborigen100500,NM,2739,33


In [61]:
# top 10 classic 
top10_classical = get_top10_players("classical")
top10_classical

,id,username,title,classical_rating,classical_progress
0,plemsovhoz,PlemSovhoz,IM,2598,-20
1,vlad_lazarev79,Vlad_Lazarev79,NaN,2476,23
2,kanake04,Kanake04,NaN,2475,13
3,ojaijoao,OjaiJoao,FM,2468,7
4,anindianchessplayer,AnIndianChessplayer,CM,2439,42
5,kc6,kc6,IM,2439,15
6,likeyacutg,likeyacutg,NaN,2436,22
7,igormezentsev,igormezentsev,FM,2435,-30
8,chesstheory64,ChessTheory64,FM,2433,46
9,teodorvasilov,TeodorVasilov,NaN,2429,-9


In [62]:
top10_ultrabullet = get_top10_players("ultraBullet")
top10_ultrabullet 

,id,username,title,ultraBullet_rating,ultraBullet_progress
0,penguingim1,penguingim1,GM,2694,36
1,blazinq,Blazinq,FM,2602,39
2,liamputnam2008,LiamPutnam2008,FM,2558,39
3,anonymousblitz,anonymousblitz,NaN,2548,-33
4,ediz_gurel,Ediz_Gurel,GM,2541,19
5,dir07,dIr07,NaN,2533,-26
6,iamstraw,iamstraw,GM,2495,-19
7,tamojerry,tamojerry,FM,2479,-4
8,afanoffun,afanoffun,CM,2478,-19
9,maybe_maybe,maybe_maybe,IM,2475,10


## Function to get latest 100 games in last 12 months

In [39]:
# last day of last month as last date 
last_date = datetime.today().replace(day=1) - timedelta(days=1) 

# date 12 months before
date_11_months_before = last_date - relativedelta(months=11)

# first day of the last 11 months as first date
first_date = date_11_months_before.replace(day=1)

start = berserk.utils.to_millis(first_date)
end = berserk.utils.to_millis(last_date)

top1classic_game = client.games.export_by_player(top10_classic["id"][0], since=start, until=end)

<generator object Games.export_by_player at 0x00000193AC67E6D0>

In [ ]:
def find_last12months_games(player_id):
    game_record100 = pd.DataFrame()
    for id in player_id :
        # list of game
        games = list(_)
    
        # add game data to dataframe 
        for i in range(100):
            try:
                game_id = games[i]["id"]

            except IndexError:
                print(f"Index {i} is out of range")

In [46]:
pd.json_normalize(top1classic_game)

""


In [40]:
games = list(_)
games[0]["createdAt"]

datetime.datetime(2024, 5, 30, 17, 19, 44, 267000, tzinfo=datetime.timezone.utc)

In [41]:
games[-1]["createdAt"]

datetime.datetime(2023, 6, 1, 17, 39, 31, 861000, tzinfo=datetime.timezone.utc)

In [44]:
game_id = games[0]["id"]
record1 = pd.json_normalize(client.games.export(game_id), max_level=1)
record1.T

,0
id,WjhujFIv
rated,True
variant,standard
speed,bullet
perf,bullet
createdAt,2024-05-30 17:19:44.267000+00:00
lastMoveAt,2024-05-30 17:22:09.930000+00:00
status,outoftime
source,pool
winner,black


In [45]:
game_id2 = games[1]["id"]
record2 = pd.json_normalize(client.games.export(game_id2), max_level=1)
record2.T

,0
id,XVHkGmfP
rated,True
variant,standard
speed,bullet
perf,bullet
createdAt,2024-05-30 17:13:47.059000+00:00
lastMoveAt,2024-05-30 17:15:59.867000+00:00
status,outoftime
source,pool
winner,white
